In [6]:
import re
from dateutil.parser import parse
import ast
import concurrent.futures
import os
import pandas as pd
import numpy as np


import itertools
import operator
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import scipy
import time


class initialEDA:
    def __init__(self, data, threadLimit=os.cpu_count() - 1, runType="initial"):
  self.data = data
  self.specNumeric = ["%", "$", "€"]
  self.threadLimit = threadLimit

    def __dType(self, row):
  print("Started __dType")
  col = row["column"]
  if row["typeDistribution"].__contains__("datetime"):
if float(row["typeDistribution"].get("datetime")) > 0.9:
    self.data[col] = self.data[col].apply(pd.to_datetime)
  print("Ended __dType")
  return self.data.dtypes[col]

    def unitType(self, data):
  try:
if self.is_date(
    data
):  # Limitation - is_date will return datetime for strings like .08
    return "datetime"
  except:
pass

  if type(data) == str:
for i in self.specNumeric:
    data = data.replace(i, " ")
  try:
k = ast.literal_eval(data)
return str(type(k).__name__)
  except:
return str(type(data).__name__)

    def __typeStringPercentage(self, row):
  print ("typeDistribution Began")
  col = row["column"]
  data = self.data[col].dropna()

  unit = data.apply(self.unitType)
  tot = data.shape[0]
  split = unit.value_counts(ascending=True).to_dict()
  for key in split:
p = (split[key] / tot).__format__(".6f")
split[key] = p

  print ("typeDistribution Ended")
  return split

    def __missing(self, row):
  col = row["column"]
  return self.data[col].isnull().sum()

    def __categorical(self, row):

  col = row["column"]
  unique = row["unique"]
  count = row["count"]
  return unique / count < 0.15

    def __unique(self, row):
  col = row["column"]
  try:
unique = self.data[col].unique()
return len(unique)
  except:
print(col, "__unique - An Exception occured")
return "NaN"

    def __count(self, row):
  col = row["column"]
  try:
return self.data[col].shape[0]
  except:
print(col, "__count - An Exception occured")
return "ERROR"

    def datatype(self, row):
  """
  This is an area of future growth. Currently the
  process uses a rule based model. But that is not ideal.
  """

  if row["raw_dType"] == "object" and row["categorical"] == False:
val = "String"
  elif row["raw_dType"] == "object" and row["categorical"] == True:
val = "Categorical"
  elif row["raw_dType"] == "object" and row["unique"] == 2:
val = "Binary"
  elif row["raw_dType"] == "int64" or row["raw_dType"] == "float64":
val = "Integer"
  elif row["raw_dType"] == "datetime64[ns]":
val = "Datetime"
  else:
val = "UNK"
  print("Ended __datatype")
  return val

    def analyze(self):
  start = time.perf_counter()

  col = self.data.columns
  eda = pd.DataFrame(col, columns=["column"])

  # Must calculate type_Distribution before the threads because otherwise it will try to end 
  eda["typeDistribution"] = eda.apply(self.__typeStringPercentage, axis=1)

  eda["raw_dType"] = eda.apply(self.__dType, axis=1)

  with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count() - 1) as executor:
# General discovery
eda["missing"] = executor.submit(eda.apply, self.__missing, axis=1).result()
eda["unique"] = executor.submit(eda.apply, self.__unique, axis=1).result()
eda["count"] = executor.submit(eda.apply, self.__count, axis=1).result()
eda["categorical"] = executor.submit(
    eda.apply, self.__categorical, axis=1
).result()

eda["data_type"] = executor.submit(
    eda.apply, self.datatype, axis=1
).result()

  finish = time.perf_counter()
  print(f"Finished in {round(finish-start, 2)/60} minutes(m)")
  return eda

In [84]:
import re
from dateutil.parser import parse
import ast
import concurrent.futures
import os
import logging

def monotone_increasing(lst):
    pairs = zip(lst, lst[1:])
    return all(itertools.starmap(operator.le, pairs))


def monotone_decreasing(lst):
    pairs = zip(lst, lst[1:])
    return all(itertools.starmap(operator.ge, pairs))


def stationary1(lst):
    stat, p, lags, obs, crit, t = adfuller(lst)
    if p > 0.05:
  return True  # ('Probably not Stationary')
    else:
  return False  # ('Probably Stationary')


def stationary2(lst):
    stat, p, lags, crit = kpss(lst)
    if p > 0.05:
  return True
    else:
  return False

    
class UnivariateAnalysis:
    def __init__(self, data, y, eda, threadCount=os.cpu_count() - 1):
  self.data = data
  self.threadCount = threadCount
  self.target = y
  self.eda = eda

    def __discrete(self, row):
  intTypes = ["float64", "int64"]
  col = row["column"]
  data = self.data[col]
  if row["raw_dType"] in intTypes:
if row["unique"] < 25:
    return True
else:
    return False

    def __ordinalDetection(self, row, threshold=0.6):

  diff = []
  ans = []
  intTypes = ["float64", "int64"]
  col = row["column"]

  if row["raw_dType"] in intTypes:
column = self.data[col]
t = column.unique().tolist()
st = sorted(t)
del t

def unique(list1):
    x = np.array(list1)
    return np.unique(x)

for i in st:
    if st.index(i) == 0:
  pass
    else:
  l = st.index(i) - 1
  f = st.index(i)
  diff.append((st[f] - st[l]))

n_diff = unique(diff)
for i in n_diff:
    answers = diff.count(i) / len(diff)
    ans.append(answers)
ma = max(ans)

if ma > threshold:
    return True
else:
    return False
  else:
return ""

    def __monotone(self, row):
  col = row["column"]
  intTypes = ["float64", "int64"]
  data = self.data[col]
  if row["raw_dType"] in intTypes:
return monotone_increasing(data) or monotone_decreasing(data)
  else:
return ""

    def __stationary(self, row):
  col = row["column"]

  intTypes = ["float64", "int64"]
  data = self.data[col].dropna()
  if row["raw_dType"] in intTypes:
return stationary1(data) or stationary2(data)
  else:
return ""

    def distributionType(self, row):
  intTypes = ["float64", "int64"]
  col = row["column"]
  if row["raw_dType"] in intTypes:

# Please write below the name of the statistical distributions that you would like to check.
# Full list is here: https://docs.scipy.org/doc/scipy/reference/stats.html
dist_names = [
    "weibull_min",
    "norm",
    "weibull_max",
    "beta",
    "invgauss",
    "uniform",
    "gamma",
    "expon",
    "lognorm",
    "pearson3",
    "triang",
]

# Read your data and set y_std to the column that you want to fit.
y_std = self.data
y_std = y_std[col].fillna("0")
y_std = pd.to_numeric(y_std)

# -------------------------------------------------
chi_square_statistics = []
size = len(y_std)

# 20 equi-distant bins of observed Data
percentile_bins = np.linspace(0, 100, 20)
percentile_cutoffs = np.percentile(y_std, percentile_bins)
observed_frequency, bins = np.histogram(y_std, bins=percentile_cutoffs)
cum_observed_frequency = np.cumsum(observed_frequency)

# Loop through candidate distributions
for distribution in dist_names:
    # Set up distribution and get fitted distribution parameters
    dist = getattr(scipy.stats, distribution)
    param = dist.fit(y_std)

    # Get expected counts in percentile bins
    # cdf of fitted sistrinution across bins
    cdf_fitted = dist.cdf(percentile_cutoffs, *param)
    expected_frequency = []
    for bin in range(len(percentile_bins) - 1):
  expected_cdf_area = cdf_fitted[bin + 1] - cdf_fitted[bin]
  expected_frequency.append(expected_cdf_area)

    # Chi-square Statistics
    expected_frequency = np.array(expected_frequency) * size
    cum_expected_frequency = np.cumsum(expected_frequency)
    ss = sum(
  ((cum_expected_frequency - cum_observed_frequency) ** 2)
  / cum_observed_frequency
    )
    chi_square_statistics.append(ss)

# Sort by minimum ch-square statistics
results = pd.DataFrame()
results["Distribution"] = dist_names
results["chi_square"] = chi_square_statistics
results.sort_values(["chi_square"], inplace=True)

return results["Distribution"].iloc[0]
  else:
return ""

    def __targetCorr(self, row):
  col = row["column"]
  correlation = 0
  intTypes = ["float64", "int64"]

  if row["raw_dType"] in intTypes:
  
while True:
    #colTarget = self.data[col].corr()[self.target][:]
    #correlation = df[col].corr()df[self.target] 
    print (row["raw_dType"])
    correlation = df[col].corr(df[self.target] )
return correlation
  else: return 0

    def analyze(self):
  start = time.perf_counter()

  col = self.data.columns

  with concurrent.futures.ThreadPoolExecutor(
max_workers=self.threadCount
  ) as executor:
# General discovery

# Classify each row into the right data type

logging.debug("__missing Start")

eda["discrete"] = executor.submit(
    eda.apply, self.__discrete, axis=1
).result()
eda["ordinal"] = executor.submit(
    eda.apply, self.__ordinalDetection, axis=1
).result()
eda["monotone"] = executor.submit(
    eda.apply, self.__monotone, axis=1
).result()
eda["stationary"] = executor.submit(
    eda.apply, self.__stationary, axis=1
).result()
eda["distribution_type"] = executor.submit(
    eda.apply, self.distributionType, axis=1
).result()

# Correlation Matrix
eda["targetCorrelation"] = executor.submit(
    eda.apply, self.__targetCorr, axis=1
).result()

# Context Generator

# Correlation to target

# Chi Squared, test of independence

# Fisher’s Exact Test of Independence

# Feature Importance

# Hoffman Correlation

  finish = time.perf_counter()
  print(f"Finished in {round(finish-start, 2)/60} minutes(m)")
  return self.eda
    
thr = os.cpu_count() - 1
UnivariateAnalysis(df,'Survived', eda, thr).analyze()



/Users/toddboetticher/anaconda3/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:956: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/var/folders/4w/fyw50xy11tldcn8ys02774dc0000gn/T/ipykernel_10776/421002212.py:27: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  stat, p, lags, crit = kpss(lst)
/var/folders/4w/fyw50xy11tldcn8ys02774dc0000gn/T/ipykernel_10776/421002212.py:27: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  stat, p, lags, crit = kpss(lst)
/var/folders/4w/fyw50xy11tldcn8ys02774dc0000gn/T/ipykernel_10776/421002212.py:27: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is 

KeyboardInterrupt: 

int64


In [46]:
import pandas as pd
df = pd.read_csv('/Users/toddboetticher/Downloads/titanic/train.csv')

In [69]:
eda = initialEDA(df).analyze()

typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
typeDistribution Began
typeDistribution Ended
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Started __dType
Ended __dType
Ended __datatype
Ended __datatype
Ended __datatype
Ended __datatype
Ended __datatype
End

In [48]:
eda

,column,typeDistribution,raw_dType,missing,unique,count,categorical,data_type
0,PassengerId,{'int': '1.000000'},int64,0,891,891,False,Integer
1,Survived,{'int': '1.000000'},int64,0,2,891,True,Integer
2,Pclass,{'int': '1.000000'},int64,0,3,891,True,Integer
3,Name,{'str': '1.000000'},object,0,891,891,False,String
4,Sex,{'str': '1.000000'},object,0,2,891,True,Categorical
5,Age,{'float': '1.000000'},float64,177,89,891,True,Integer
6,SibSp,{'int': '1.000000'},int64,0,7,891,True,Integer
7,Parch,{'int': '1.000000'},int64,0,7,891,True,Integer
8,Ticket,"{'str': '0.258137', 'int': '0.741863'}",object,0,681,891,False,String
9,Fare,{'float': '1.000000'},float64,0,248,891,False,Integer


In [70]:
UnivariateAnalysis(df,'Survived', eda).analyze()

/Users/toddboetticher/anaconda3/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:956: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/var/folders/4w/fyw50xy11tldcn8ys02774dc0000gn/T/ipykernel_10776/1717585814.py:27: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  stat, p, lags, crit = kpss(lst)
/var/folders/4w/fyw50xy11tldcn8ys02774dc0000gn/T/ipykernel_10776/1717585814.py:27: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  stat, p, lags, crit = kpss(lst)
/var/folders/4w/fyw50xy11tldcn8ys02774dc0000gn/T/ipykernel_10776/1717585814.py:27: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value 

TypeError: Series.corr() missing 1 required positional argument: 'other'

In [79]:
#col = row["column"]
df['Fare'].corr(df['Survived'])

0.2573065223849622

In [60]:
eda


,column,typeDistribution,raw_dType,missing,unique,count,categorical,data_type,discrete,ordinal,monotone,stationary,distribution_type
0,PassengerId,{'int': '1.000000'},int64,0,891,891,False,Integer,False,True,True,True,uniform
1,Survived,{'int': '1.000000'},int64,0,2,891,True,Integer,True,True,False,True,weibull_min
2,Pclass,{'int': '1.000000'},int64,0,3,891,True,Integer,True,True,False,True,weibull_min
3,Name,{'str': '1.000000'},object,0,891,891,False,String,None,,,,
4,Sex,{'str': '1.000000'},object,0,2,891,True,Categorical,None,,,,
5,Age,{'float': '1.000000'},float64,177,89,891,True,Integer,False,False,False,True,weibull_min
6,SibSp,{'int': '1.000000'},int64,0,7,891,True,Integer,True,True,False,True,weibull_min
7,Parch,{'int': '1.000000'},int64,0,7,891,True,Integer,True,True,False,True,weibull_min
8,Ticket,"{'str': '0.258137', 'int': '0.741863'}",object,0,681,891,False,String,None,,,,
9,Fare,{'float': '1.000000'},float64,0,248,891,False,Integer,False,False,False,True,weibull_min
